# Code for visualizing Plenoxel voxel

In [20]:
path_to_ckpt_dir = ""
ckpt_dirs = []
# path_to_ckpt_dir = "/home/yoonwoo/data/perfception/perfception_co3d"
# ckpt_dirs = ["plenoxel_torch_110_13051_23361"]
path_to_ckpt_dir = "/home/yoonwoo/Workspace/NeRF-Factory/logs"
ckpt_dirs = ["plenoxel_co3d_147_16372_32078_test"]

In [21]:
import os
os.environ['DISPLAY']=":1"

import numpy as np
import open3d as o3d
import torch

In [22]:
def voxel2pcd(ckpt_dir):

    path_candidate = os.path.join(ckpt_dir, "last.ckpt")
    if not os.path.exists(path_candidate):
        path_candidate = os.path.join(ckpt_dir, "data.ckpt")
    ckpt = torch.load(path_candidate)

    density = ckpt["state_dict"]["model.density_data"].detach().cpu()
    links_idx = ckpt["state_dict"]["model.links_idx"].detach().cpu()
    valid = torch.where(density > 0.0)[0].long()
    density, links_idx = density[valid], links_idx[valid].long()

    resolution = (
        ckpt["reso_list"]["reso_idx"] 
        if "reso_list" in ckpt.keys() else 
        # [256, 256, 256]
        [128, 128, 128]
    )
    
    links_idx = torch.stack(
        [
            links_idx // (resolution[1] * resolution[2]),
            links_idx % (resolution[1] * resolution[2]) // resolution[2],
            -links_idx % resolution[2],
        ],
        -1,
    )
    pts = links_idx.numpy().astype(np.float64)
    pts_color = (density - density.min()) / (density.max() - density.min())
    pts_color = pts_color.numpy().astype(np.float64).repeat(3, axis=-1)

    pts = np.concatenate([pts], axis=0)
    pts_color = np.concatenate([pts_color], axis=0)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    pcd.colors = o3d.utility.Vector3dVector(pts_color)
    o3d.visualization.draw_geometries([pcd])

In [23]:
for ckpt_dir in ckpt_dirs: 
    path_ckpt_dir = os.path.join(path_to_ckpt_dir, ckpt_dir)
    voxel2pcd(path_ckpt_dir)